# STUNIR Getting Started Tutorial

Welcome to STUNIR! This interactive notebook will guide you through the basics of using STUNIR for deterministic code generation and verification.

## What You'll Learn

1. **Core Concepts**: Understanding specs, IR, and receipts
2. **Basic Operations**: Loading specs and generating IR
3. **Receipts**: Generating and verifying build receipts
4. **Determinism**: Verifying deterministic output

## 1. Setup and Imports

First, let's import the necessary modules:

In [ ]:
import json
import hashlib
from datetime import datetime, timezone
from typing import Dict, Any, List

# Helper functions
def canonical_json(data: Any) -> str:
    """Generate RFC 8785 compliant canonical JSON."""
    return json.dumps(data, sort_keys=True, separators=(',', ':'), ensure_ascii=False)

def compute_sha256(data: str) -> str:
    """Compute SHA-256 hash."""
    return hashlib.sha256(data.encode('utf-8')).hexdigest()

print('✅ Setup complete!')

## 2. Understanding Specs

A **spec** is the input to STUNIR. It defines:
- Module name and version
- Functions with parameters and return types
- Exports (public API)

Let's create a simple spec:

In [ ]:
# Define a simple spec
spec = {
    'name': 'calculator',
    'version': '1.0.0',
    'description': 'A simple calculator module',
    'functions': [
        {
            'name': 'add',
            'params': [
                {'name': 'a', 'type': 'i32'},
                {'name': 'b', 'type': 'i32'}
            ],
            'returns': 'i32'
        },
        {
            'name': 'subtract',
            'params': [
                {'name': 'a', 'type': 'i32'},
                {'name': 'b', 'type': 'i32'}
            ],
            'returns': 'i32'
        }
    ],
    'exports': ['add', 'subtract']
}

print('Spec created:')
print(json.dumps(spec, indent=2))

## 3. Converting Spec to IR

The **Intermediate Representation (IR)** is a normalized, canonical form of the spec. Key properties:
- Deterministic structure
- Includes computed metadata (hashes, epochs)
- Ready for code generation

In [ ]:
def spec_to_ir(spec: Dict[str, Any]) -> Dict[str, Any]:
    """Convert spec to IR."""
    # Compute spec hash
    spec_json = canonical_json(spec)
    spec_hash = compute_sha256(spec_json)
    
    # Build IR
    ir = {
        'ir_version': '1.0.0',
        'ir_epoch': int(datetime.now(timezone.utc).timestamp()),
        'ir_spec_hash': spec_hash,
        'module': {
            'name': spec['name'],
            'version': spec['version']
        },
        'functions': [],
        'exports': spec.get('exports', [])
    }
    
    # Transform functions
    for func in spec.get('functions', []):
        ir_func = {
            'name': func['name'],
            'signature': {
                'params': func.get('params', []),
                'returns': func.get('returns', 'void')
            }
        }
        ir['functions'].append(ir_func)
    
    return ir

# Convert our spec
ir = spec_to_ir(spec)
print('Generated IR:')
print(json.dumps(ir, indent=2))

## 4. Generating Receipts

**Receipts** provide cryptographic proof of build artifacts:
- Hash of the IR
- Hash of the original spec
- Self-referential receipt hash

This enables verification and auditing.

In [ ]:
def generate_receipt(ir: Dict[str, Any]) -> Dict[str, Any]:
    """Generate a receipt for IR."""
    ir_json = canonical_json(ir)
    ir_hash = compute_sha256(ir_json)
    
    receipt = {
        'receipt_version': '1.0.0',
        'receipt_epoch': int(datetime.now(timezone.utc).timestamp()),
        'module_name': ir['module']['name'],
        'ir_hash': ir_hash,
        'spec_hash': ir['ir_spec_hash'],
        'function_count': len(ir['functions'])
    }
    
    # Compute receipt hash
    content = canonical_json({k: v for k, v in receipt.items()})
    receipt['receipt_hash'] = compute_sha256(content)
    
    return receipt

# Generate receipt
receipt = generate_receipt(ir)
print('Generated Receipt:')
print(json.dumps(receipt, indent=2))

## 5. Verifying Determinism

A key feature of STUNIR is **determinism** - the same input always produces the same output.

Let's verify this:

In [ ]:
def verify_determinism(spec: Dict[str, Any], iterations: int = 5) -> bool:
    """Verify deterministic output."""
    hashes = []
    
    for i in range(iterations):
        json_str = canonical_json(spec)
        hash_val = compute_sha256(json_str)
        hashes.append(hash_val)
        print(f'  Iteration {i+1}: {hash_val[:32]}...')
    
    is_deterministic = len(set(hashes)) == 1
    return is_deterministic

print('Verifying determinism...')
result = verify_determinism(spec)
print(f'\n✅ Determinism verified!' if result else '\n❌ Determinism failed!')

## 6. Summary

In this tutorial, you learned:

1. **Specs** define your module's structure
2. **IR** is the canonical intermediate form
3. **Receipts** provide cryptographic verification
4. **Determinism** ensures reproducible builds

### Next Steps

- Explore `advanced_features.ipynb` for multi-target generation
- Check out the Python examples in `examples/python/`
- Read the full documentation at `docs/`

In [ ]:
# Final summary
print('=' * 50)
print('Tutorial Complete!')
print('=' * 50)
print(f'Module:        {spec["name"]}')
print(f'Functions:     {len(spec["functions"])}')
print(f'IR Hash:       {receipt["ir_hash"][:32]}...')
print(f'Receipt Hash:  {receipt["receipt_hash"][:32]}...')